In [58]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [59]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [60]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [61]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/ENRIQUEZ Acumulado de pagos Diciembre de 2020.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [62]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS DIC LIVERPOOL .xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
try: 
    pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
except: pagos_liv['Fecha Pago']   
pagos_liv['Fecha Pago'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Pago']]
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom']    
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
#pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [63]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [64]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [65]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [66]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

In [67]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

# Llamadas detallado Liverpool Bradesco

In [68]:
#Hacemos la consulta referente a las llamadas de Liverpool
llamadas_liv = consulta_sql('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^LIV)')
llamadas_liv.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_liv.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1606804940.8041242,Inbound,2020-12-01 00:42:21,1102_IN,LIVERPOOL_BLASTER,999,Default inbound list,27470625,5591048001,...,0 days 00:42:21,0,00:00 - 01:00,ANSWER,16,200 OK,INBOUND,559,CDMX,CENTRO


In [69]:
#Hacemos la consulta referente a las llamadas de Bradesco
llamadas_brad = consulta_sql('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^BRAD)')
llamadas_brad.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_brad.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1606818712.8041254,Inbound,2020-12-01 04:31:53,1001_IN,BRADESCO_CALL,999,Default inbound list,16344681,MCM Telecom,...,0 days 04:31:53,4,04:00 - 05:00,ANSWER,16,200 OK,INBOUND,555,CDMX,CENTRO


In [70]:
llamadas_liv_brad = pd.concat([llamadas_liv,llamadas_brad],ignore_index=True)

In [71]:
llamadas_liv_brad['turno'] = np.where(llamadas_liv_brad['call_time_hour']<=14,'M','V')

In [72]:
#Obtenemos el performance bruto de cada gestor
control_gestores_bruto_brad = pd.pivot_table(llamadas_liv_brad,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_bruto_brad = to_dataframe(control_gestores_bruto_brad)
control_gestores_bruto_brad = control_gestores_bruto_brad.iloc[:,[0,1,4,2,3]]
control_gestores_bruto_brad.columns = ['Gestor','Turno','NumeroGestionesB','TiempoTotalSegB','PromedioLlamadaSegB']
control_gestores_bruto_brad['TiempoTotalMinB'] = [x/60 for x in control_gestores_bruto_brad['TiempoTotalSegB']]
control_gestores_bruto_brad

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB
0,,M,3,0,0.000000,0.000000
1,AGUILAR MACEDO VALERIA,M,1460,61900,42.397260,1031.666667
2,AGUILAR MACEDO VALERIA,V,495,14032,28.347475,233.866667
3,ALANIZ DIAZ ALISON JOCELIN,M,3922,114054,29.080571,1900.900000
4,ALANIZ DIAZ ALISON JOCELIN,V,694,17690,25.489914,294.833333
...,...,...,...,...,...,...
116,VAZQUEZ SANCHEZ LINNETH BERENICE,V,533,39019,73.206379,650.316667
117,VEGA HERNANDEZ NATALIE,M,2346,160689,68.494885,2678.150000
118,VEGA HERNANDEZ NATALIE,V,674,40931,60.728487,682.183333
119,VELAZQUEZ IBARRA ERICK ALEJANDRO,M,3679,152688,41.502582,2544.800000


In [73]:
#Obtenemos el performance de las llamadas contestadas
llamadas_liv_brad2 = llamadas_liv_brad.loc[llamadas_liv_brad['status_name'].isin(['Contacto con titular','Promesa de pago','Mensaje con terceros','Mensaje con familiar','Negativa de pago','Contacto familiar','Agendar llamada'])]
control_gestores_llamadas_brad = pd.pivot_table(llamadas_liv_brad2,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_llamadas_brad = to_dataframe(control_gestores_llamadas_brad)
control_gestores_llamadas_brad = control_gestores_llamadas_brad.iloc[:,[0,1,4,2,3]]
control_gestores_llamadas_brad.columns = ['Gestor','Turno','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_llamadas_brad['TiempoTotalMin'] = [x/60 for x in control_gestores_llamadas_brad['TiempoTotalSeg']]
#control_gestores_llamadas_brad

In [74]:
control_liv_consol_turno = pd.merge(control_gestores_bruto_brad,control_gestores_llamadas_brad,how='left',on=['Gestor','Turno'])
control_liv_consol_turno.fillna(0,inplace=True)
control_liv_consol_turno.head(2)

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,,M,3,0,0.00000,0.000000,0.0,0.0,0.000000,0.0
1,AGUILAR MACEDO VALERIA,M,1460,61900,42.39726,1031.666667,45.0,6072.0,134.933333,101.2


In [75]:
#Obtenemos el performance de las llamadas contestadas
control_gestores_total = pd.pivot_table(llamadas_liv_brad2,index=['full_name'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_total = to_dataframe(control_gestores_total)
control_gestores_total = control_gestores_total.iloc[:,[0,3,1,2]]
control_gestores_total.columns = ['Gestor','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_total['TiempoTotalMin'] = [x/60 for x in control_gestores_total['TiempoTotalSeg']]
control_gestores_total

,Gestor,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,AGUILAR MACEDO VALERIA,73,8658,118.602740,144.300000
1,ALANIZ DIAZ ALISON JOCELIN,156,21753,139.442308,362.550000
2,ALEJANDRA YESENIA HERNANDEZ BITE,21,3055,145.476190,50.916667
3,BALBUENA MACEDO EDGAR URIEL,100,20141,201.410000,335.683333
4,BARRETO OSEGUERA MARIA FERNANDA,147,29402,200.013605,490.033333
5,BAUTISTA HILARIO YANELY,45,10640,236.444444,177.333333
6,BERENICE GUZMAN JAIME,24,4743,197.625000,79.050000
7,CARRANCO ZAMORA JAIME WALBERTO,185,29079,157.183784,484.650000
8,CASTRO REZA IRVING YAEL,116,17061,147.077586,284.350000
9,CID IRINEO ROSA ISELA,218,35719,163.848624,595.316667


# Performance Liverpool Bradesco

In [76]:
#Hacemos la consulta referente al performance del Gestor
performance_all = consulta_sql('procesos_externos.all_rpt_agent_performance',mes,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [77]:
performance_liv_brad = performance_all.loc[(performance_all['user_group']=='LIVERPOOL') | (performance_all['user_group']=='BRADESCARD')]
performance_liv_brad[['ocupacion','adherencia']] = percent_tonumeric(performance_liv_brad,['ocupacion','adherencia'])

In [78]:
def to_hour(df,cols):
    df[cols] = [datetime.strptime(x,'%H:%M:%S') for x in df[cols]]
    return df
    
#performance_liv_brad['login_vici2'] = to_hour(performance_liv_brad,'login_vici')
#performance_liv_brad['logout_vici2'] = to_hour(performance_liv_brad,'logout_vici')

In [79]:
performance_liv_brad['hora_login'] = [int(x[:2]) for x in performance_liv_brad['login_vici']]
performance_liv_brad['hora_logout'] = [int(x[:2]) for x in performance_liv_brad['logout_vici']]
performance_liv_brad['turno'] = np.where((performance_liv_brad['hora_logout']<=14)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'M',
                                    np.where((performance_liv_brad['hora_logout']>17)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'V','Mixto'))

In [80]:
performance_liv_brad['horas_trabajadas'] = performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas']/performance_liv_brad['horas_trabajadas']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas_hora'].round(2)
performance_liv_brad.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
1,2020-12-01,AGUILARMAVA,AGUILAR MACEDO VALERIA,BRADESCARD,64,00:00:46,9.83,14.93,13:05:58,20:57:22,...,00:00:00,00:00:00,00:00:53,00:01:32,00:04:04,13,20,V,7,9.14


In [81]:
control_performance_brad_turno = pd.pivot_table(performance_liv_brad,index=['nombre_usuario','turno'],values=['llamadas','ocupacion','llamadas_hora'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_brad_turno = to_dataframe(control_performance_brad_turno)
control_performance_brad_turno = control_performance_brad_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
control_performance_brad_turno.columns = ['Gestor','Turno','SumLlamadasDia','SumLlamadasHora','SumOcupacion','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion','DiasAsitencia','MinLlamadasDia','MinLlamadasHora','MinOcupacion','MaxLlamadasDia','MaxLlamadasHora','MaxOcupacion']
control_performance_brad_turno.head(1)

,Gestor,Turno,SumLlamadasDia,SumLlamadasHora,SumOcupacion,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion,DiasAsitencia,MinLlamadasDia,MinLlamadasHora,MinOcupacion,MaxLlamadasDia,MaxLlamadasHora,MaxOcupacion
0,AGUILAR MACEDO VALERIA,M,612,104.05,80.67,102.0,17.341667,13.445,6,58,11.67,6.87,142,22.33,21.02


In [82]:
control_performance_brad_turno2 = control_performance_brad_turno[['Gestor','Turno','PromedioLlamadasHora','PromedioOcupacion']]

In [83]:
control_performance_brad = pd.pivot_table(performance_liv_brad,index=['nombre_usuario'],values=['llamadas','ocupacion','llamadas_hora','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_brad = to_dataframe(control_performance_brad)
control_performance_brad = control_performance_brad.iloc[:,[0,1,2,6,7,8]]
control_performance_brad.columns = ['Gestor','SumHorasTrabajadas','SumLlamadasDia','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion']
control_performance_brad.head(3)

,Gestor,SumHorasTrabajadas,SumLlamadasDia,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion
0,AGUILAR MACEDO VALERIA,132,1955,130.333333,15.014000,11.082000
1,ALANIZ DIAZ ALISON JOCELIN,108,4616,329.714286,43.137143,32.732857
2,ALEJANDRA YESENIA HERNANDEZ BITE,6,113,113.000000,18.830000,20.140000


In [84]:
conjunto_performance_llamadas = pd.merge(control_gestores_total,control_performance_brad,on='Gestor',how='outer')
conjunto_performance_llamadas['ContactoHora'] = conjunto_performance_llamadas['NumeroGestiones']/conjunto_performance_llamadas['SumHorasTrabajadas']
conjunto_performance_llamadas = conjunto_performance_llamadas.iloc[:,[0,8,10,9,]]
conjunto_performance_llamadas.head(3)

,Gestor,PromedioLlamadasHora,ContactoHora,PromedioOcupacion
0,AGUILAR MACEDO VALERIA,15.014000,0.553030,11.082000
1,ALANIZ DIAZ ALISON JOCELIN,43.137143,1.444444,32.732857
2,ALEJANDRA YESENIA HERNANDEZ BITE,18.830000,3.500000,20.140000


# Gestion detallado Liverpool Bradesco

# Liverpool

In [85]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv = consulta_sql('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])
gestion_liv['anio'] = [int(x[:4]) for x in gestion_liv['fecha_promesa']]
gestion_liv = gestion_liv.loc[(gestion_liv['anio']>2010)&(gestion_liv['anio']<2100)]

In [86]:
#Hacemos la consulta de la asignacion
asignacion_liv = consulta_sql('liverpool_rpt_asignacion_activa')
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])

In [87]:
#Hacemos el consolidado 
consolidado_liv_mes = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes.fillna('COBRANZA',inplace=True)
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])
#consolidado_liv_mes.head(3)

In [88]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']
#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]

In [89]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
try:consolidado_pagos_liv['fecha_promesa'] = pd.to_datetime(consolidado_pagos_liv['fecha_promesa'],dayfirst=False)
except: consolidado_pagos_liv['fecha_promesa'] = consolidado_pagos_liv['fecha_promesa']
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']
#consolidado_pagos_liv.head(3)

In [90]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]
#consolidado_pagos_liv.head(3)

In [91]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [92]:
#Obtenemos datos de los gestores de Liverpool
prom_liv_gestor_mes = pd.pivot_table(consolidado_pagos_liv,index=['nombre_usuario'],values=['credito','monto_promesa','PagoReal','MontoArancel'],aggfunc=['count',np.sum],dropna=False)
prom_liv_gestor_mes = pd.DataFrame(prom_liv_gestor_mes.to_records())
prom_liv_gestor_mes = prom_liv_gestor_mes.iloc[:,[0,2,1,8,6,5]]
prom_liv_gestor_mes.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
prom_liv_gestor_mes.fillna(0,inplace=True)
#prom_liv_gestor_mes = prom_liv_gestor_mes.sort_values(by='SumaPagos',ascending=False)
#prom_liv_gestor_mes[['SumaPromesas']] = prom_liv_gestor_mes[['SumaPromesas']].applymap("{0:.2f}".format)
prom_liv_gestor_mes.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel
0,BALBUENA MACEDO EDGAR URIEL,1,0,14938.0,0.0,0.0


# Bradesco

In [93]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = consulta_sql('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['credito'] = pd.to_numeric(gestion_brad_mes['credito'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])
#gestion_brad_mes.head(3)

In [94]:
#Hacemos la consulta de la asignacion
asignacion_brad = consulta_sql('liverpool_rpt_asignacion_activa')
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,9,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','saldo_total','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
#asignacion_brad.head(3)

In [95]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
consolidado_brad_mes.fillna('COBRANZA',inplace=True)
#consolidado_brad_mes.head(3)

In [96]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [97]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [98]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [99]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]

In [100]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad2,how='left',left_on=['credito'],right_on=['NUMERO_TARJETA'])
try: consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)
except: consoliado_pagos_brad['fecha_promesa'] = consoliado_pagos_brad['fecha_gestion']

In [101]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [102]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['credito','MONTO','FECHA_PAGO','usuario'])

In [103]:
pagos_totales_brad2 = pd.pivot_table(consoliado_pagos_brad,index=['nombre_usuario'],values=['monto_promesa','MONTO','PagoReal'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,3,1,6,4,5]]
pagos_totales_brad2.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
pagos_totales_brad2.fillna(0,inplace=True)
#pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel
0,AGUILAR MACEDO VALERIA,129,86,145456.0,116867.0,12817.57


In [104]:
gestor_brad_liv = pd.concat([pagos_totales_brad2,prom_liv_gestor_mes],ignore_index=True)
gestor_final_brad_liv = pd.pivot_table(gestor_brad_liv,index=['Gestor'],values=['NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel'],aggfunc=np.sum)
gestor_final_brad_liv = pd.DataFrame(gestor_final_brad_liv.to_records())
gestor_final_brad_liv = gestor_final_brad_liv.iloc[:,[0,2,1,5,4,3]]
gestor_final_brad_liv['EfVolumen'] = gestor_final_brad_liv['NumeroPagos']/gestor_final_brad_liv['NumeroPromesas']
gestor_final_brad_liv['EfPagos'] = gestor_final_brad_liv['SumaPagos']/gestor_final_brad_liv['SumaPromesas']
gestor_final_brad_liv.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel,EfVolumen,EfPagos
0,AGUILAR MACEDO VALERIA,129,86,145456.0,116867.0,12817.57,0.666667,0.803453


In [105]:
gestor_final_brad_liv2 = gestor_final_brad_liv[['Gestor','SumaPagos','EfPagos']]
gestor_final_brad_liv2.head(1)

,Gestor,SumaPagos,EfPagos
0,AGUILAR MACEDO VALERIA,116867.0,0.803453


In [106]:
final = pd.merge(gestor_final_brad_liv2,conjunto_performance_llamadas,how='outer',on='Gestor')
final['PromedioaLlamadasHora'] = final['PromedioLlamadasHora'].round()
final['PromedioOcupacion'] = final['PromedioOcupacion']/100
cols = ['PromedioOcupacion','EfPagos','PromedioLlamadasHora','0.ContactoHora']
#final[cols] = final[cols].round(2)
final.fillna(0,inplace=True)
final

,Gestor,SumaPagos,EfPagos,PromedioLlamadasHora,ContactoHora,PromedioOcupacion,PromedioaLlamadasHora
0,AGUILAR MACEDO VALERIA,116867.0,0.803453,15.014000,0.553030,0.110820,15.0
1,ALANIZ DIAZ ALISON JOCELIN,93757.0,0.516952,43.137143,1.444444,0.327329,43.0
2,ALEJANDRA YESENIA HERNANDEZ BITE,9246.0,0.389469,18.830000,3.500000,0.201400,19.0
3,BALBUENA MACEDO EDGAR URIEL,43437.0,0.492712,21.578462,0.925926,0.432308,22.0
4,BARRETO OSEGUERA MARIA FERNANDA,69327.0,0.498375,19.506364,2.013699,0.217809,20.0
...,...,...,...,...,...,...,...
59,VELAZQUEZ IBARRA ERICK ALEJANDRO,45542.0,0.460157,0.000000,0.000000,0.000000,0.0
60,LOPEZ VAZQUEZ ERIK,0.0,0.000000,23.223333,1.562500,0.295700,23.0
61,VAZQUEZ SANCHEZ LINNETH BERENICE,0.0,0.000000,22.380667,0.475248,0.340113,22.0
62,VELAZQUEZ IBARRA ERICK ALEJANDRO,0.0,0.000000,inf,0.370968,0.441600,inf


In [107]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/BradLiv/Reporte Gestor BradLiv '+now+'.xlsx',engine='xlsxwriter')
final.to_excel(writer,'BSC',index=False,header=True)
gestor_final_brad_liv2.to_excel(writer,'MontoRecuperado',index=False,header=True)
control_liv_consol_turno.to_excel(writer,'LlamadasDetallado',index=False,header=True)
control_performance_brad_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
gestor_final_brad_liv.to_excel(writer,'PagosDetallado',index=False,header=True)

writer.save()